In [ ]:
import time as time
import pandas as pd
import os

from selenium import webdriver 

In [ ]:
urls = ['http://www.bcpharmacists.org/list-community-pharmacies', 
        'http://www.bcpharmacists.org/list-hospital-pharmacies']
df = pd.DataFrame()

In [ ]:
# Hepler function that saves the data to disk by appending
def appendToFile(filePath, df):
    header = True    
    # If file exist don't append the header 
    if os.path.isfile(filePath):
        header = False
        
    df.to_csv(filePath, mode='a', header=header, index=False)

In [ ]:
option = webdriver.ChromeOptions()
option.add_argument('-incognito')

browser = webdriver.Chrome(executable_path='../chromedriver', chrome_options=option)

In [ ]:
for url in urls:
    # Go to url
    browser.get(url)

    # Wait 2 seconds for the url to load the entire list
    time.sleep(2)

    # Grab table element with the pharmacies
    table = browser.find_element_by_xpath("//table[@class='sticky-enabled']")

    # Find all the table rows in the table
    trList = table.find_elements_by_tag_name('tr')

    # Remove header row
    trList = trList[1:]
    
    # For each row, scrape the relevant infos
    for tr in trList:
        # Grab all the table cells in the row
        td = tr.find_elements_by_tag_name('td')
        
        # Repackage those cells into a dictionary
        data = {'Pharmacy': td[0].text,
              'Address': td[1].text,
              'Manager': td[2].text,
              'Phone': td[3].text,
              'Fax': td[4].text}
        
        # Append the data into the global dataframe
        df = df.append(data, ignore_index=True)

In [ ]:
df.to_csv('./csv/BC Pharmacy.csv', index=False)